# CT for GATE 

In [3]:
import os
import SimpleITK as sitk
import numpy as np
import array

import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interact

In [4]:
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.gray)
    interact(plot_image, myslice=(0, im.shape[0]))
    return; 

In [5]:
def ct_crop(pix, x1crop, x2cropt, y1crop, y2cropt):
    print 'ORIGINAL : '
    ct_viewer(pix)
    x2crop=x2cropt-pix.shape[1]
    y2crop=y2cropt-pix.shape[2]
    pix_crop=pixel_array[:, x1crop:x2crop, y1crop:y2crop]
    print 'CROPPED :'
    ct_viewer(pix_crop)
    return pix_crop;


## read  dicom images

In [10]:
#read dcm directory and compte ONE sitk 3D image object
reader=sitk.ImageSeriesReader()
#dicom_name=reader.GetGDCMSeriesFileNames('scanX/')
dicom_name=reader.GetGDCMSeriesFileNames('CTscanElec')
reader.SetFileNames(dicom_name)
ct_image = reader.Execute()

print(ct_image.GetOrigin())
dicom_ct_dimension=ct_image.GetSize()
dicom_ct_pixel_size=ct_image.GetSpacing()
print('Dimesions : ' , dicom_ct_dimension)
print('Pixels Size :' , dicom_ct_pixel_size)

x0=0.5*dicom_ct_dimension[0]*dicom_ct_pixel_size[0]
y0=-0.5*dicom_ct_dimension[1]*dicom_ct_pixel_size[1]
z0=-0.5*dicom_ct_dimension[2]*dicom_ct_pixel_size[2]

print x0
print y0
print z0

pixel_array=sitk.GetArrayFromImage(ct_image)   #Get numpy n-d array from sitk object to plot it                          
ct_viewer(pixel_array)

(-325.0, -325.0, -180.0)
('Dimesions : ', (512, 512, 161))
('Pixels Size :', (1.269531, 1.269531, 2.5))
324.999936
-324.999936
-201.25


interactive(children=(IntSlider(value=80, description=u'myslice', max=161), Output()), _dom_classes=('widget-interact',))

## create raw image fom CT dicom

In [ ]:
data = pixel_array.flatten()

#Write Raw File 
filename = 'ct_patient_test'
binfile = filename + '.raw'
bitas = array.array('h', data)
print binfile 
with open(binfile, 'wb') as file:
    file.write(bitas)

#Write mhd File
mhdFile = filename + '.mhd'
print mhdFile

with open(mhdFile, 'wt') as file:
    file.write('ObjectType = Image\n')
    file.write('NDims = ' + str(len(dicom_ct_dimension)) + '\n')
    file.write('BinaryData = True\n')
    file.write('BinaryDataByteOrderMSB = False\n')
    file.write('CompressedData = False\n')
    file.write('TransformMatrix = 1 0 0 0 1 0 0 0 1\n')
    file.write('Offset = 0 0 0\n')
    file.write('CenterOfRotation = 0 0 0\n')
    file.write('ElementSpacing = ' + str(dicom_ct_pixel_size[0]) +' '+ str(dicom_ct_pixel_size[1]) +' '+ str(dicom_ct_pixel_size[2]) +'\n')
    file.write('DimSize = ' + str(dicom_ct_dimension[0]) +' '+ str(dicom_ct_dimension[1]) +' '+ str(dicom_ct_dimension[2]) + '\n')
    file.write('ElementType = MET_SHORT\n')
    file.write('ElementDataFile = ' + binfile)


## crop CT 


In [6]:
#select apropriates value to crop scan 
#use sliders to view all the slice and refer to original ct


y1=100 #   X axis lower value (on original ct)
y2=370  #   X axis upper value (on original ct)
x1=150  #   Y axis lower value (on original ct)
x2=370  #   Y axis upper value (on original ct)


pixel_array_crop=ct_crop(pixel_array, y1, y2, x1, x2);

ORIGINAL : 


interactive(children=(IntSlider(value=89, description=u'myslice', max=178), Output()), _dom_classes=('widget-interact',))

CROPPED :


interactive(children=(IntSlider(value=89, description=u'myslice', max=178), Output()), _dom_classes=('widget-interact',))

In [ ]:
print pixel_array_crop.shape
a=pixel_array_crop.shape[2]*dicom_ct_pixel_size[0]/5
b=pixel_array_crop.shape[1]*dicom_ct_pixel_size[1]/5
c=pixel_array_crop.shape[0]*dicom_ct_pixel_size[2]/5
print a, b, c

## create raw image from cropped CT 

In [ ]:
data = pixel_array_crop.flatten()

#Write Raw File 
filename_crop = 'ct_patient_test' + '_crop' 
binfile = filename_crop + '.raw'
bitas = array.array('h', data)
print binfile 
with open(binfile, 'wb') as file:
    file.write(bitas)

#Write mhd File
mhdFile = filename_crop + '.mhd'
print mhdFile

with open(mhdFile, 'wt') as file:
    file.write('ObjectType = Image\n')
    file.write('NDims = ' + str(len(dicom_ct_dimension)) + '\n')
    file.write('BinaryData = True\n')
    file.write('BinaryDataByteOrderMSB = False\n')
    file.write('CompressedData = False\n')
    file.write('TransformMatrix = 1 0 0 0 1 0 0 0 1\n')
    file.write('Offset = 0 0 0\n')
    file.write('CenterOfRotation = 0 0 0\n')
    file.write('ElementSpacing = ' + str(dicom_ct_pixel_size[0]) +' '+ str(dicom_ct_pixel_size[1]) +' '+ str(dicom_ct_pixel_size[2]) +'\n')
    file.write('DimSize = ' + str(pixel_array_crop.shape[2]) +' '+ str(pixel_array_crop.shape[1]) +' '+ str(pixel_array_crop.shape[0]) + '\n')
    file.write('ElementType = MET_SHORT\n')
    file.write('ElementDataFile = ' + binfile)


## Isocenter position 

In [ ]:
#iso rt plan 
iso_rt_x = 1.245
iso_rt_y = -138.337
iso_rt_z = 2.5


#calcul de la position de l'origine de ref du ct original dans le repére GATE (*-1 car rotation suivant l'axe x dans macro GATE)
origin_img_0_x= (-0.5*dicom_ct_dimension[0]*dicom_ct_pixel_size[0])
origin_img_0_y= (0.5*dicom_ct_dimension[1]*dicom_ct_pixel_size[1])
origin_img_0_z= (0.5*dicom_ct_dimension[2]*dicom_ct_pixel_size[2])


#calcul de la position de l'isocentre du ct original dans le repére ct a
iso_a_x= iso_rt_x - origin_img_0_x
iso_a_y= iso_rt_y - origin_img_0_y
iso_a_z= iso_rt_z - origin_img_0_z


# prise en compte de la rotation calcul du symetrique (vol center in ct a ref):
vol_center_x=0.5*dicom_ct_dimension[0]*dicom_ct_pixel_size[0]
vol_center_y=-0.5*dicom_ct_dimension[1]*dicom_ct_pixel_size[1]
vol_center_z=-0.5*dicom_ct_dimension[2]*dicom_ct_pixel_size[2]  

#calcul du symetrique
x_iso=(2*vol_center_x)-iso_a_x
y_iso=(2*vol_center_y)-iso_a_y
z_iso=(2*vol_center_z)-iso_a_z


print 'If native scan is used for GATE simulation use this value with /translateTheImageAtThisIsocenter :'
print x_iso, y_iso, z_iso
print
print


#for cropped scan

#origin ref ct b
origin_img_1_x=(x1)*dicom_ct_pixel_size[0]
origin_img_1_y=(-y1)*dicom_ct_pixel_size[0]
origin_img_1_z= origin_img_0_z               # car croppage en x et y 

x_iso_crop = x_iso - origin_img_1_x
y_iso_crop = y_iso - origin_img_1_y
z_iso_crop = z_iso
print 'If CROP scan is used for GATE simulation use this value with /translateTheImageAtThisIsocenter :'
print x_iso_crop, y_iso_crop, z_iso_crop